**Table of contents**<a id='toc0_'></a>    
- [Projet 4 Openclassrooms : Construisez un modèle de scoring](#toc1_)    
  - [Présentation du projet](#toc1_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Projet 4 Openclassrooms : Construisez un modèle de scoring](#toc0_)

![Description de l'image](https://user.oc-static.com/upload/2023/03/24/16796540347308_Data%20Scientist-P7-01-banner.png)

## <a id='toc1_1_'></a>[Présentation du projet](#toc0_)

Vous êtes Data Scientist au sein d'une société financière, nommée "Prêt à dépenser",  qui propose des crédits à la consommation pour des personnes ayant peu ou pas d'historique de prêt.<br>
Elle souhaite donc développer un **algorithme de classification** pour aider à décider si un prêt peut être accordé à un client.

## Installation des librairies

In [1]:
# Import library
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import pandas as pd
import os
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
import missingno as msno
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Pandas options display
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

In [ ]:
# Seaborn theme

sns.set_theme(style="darkgrid")
plt.rcParams.update(
    {
        'xtick.labelsize': 25,
        'ytick.labelsize': 25,
        'axes.labelsize': 25,
        'legend.fontsize': 25,
        'axes.titlesize': 45,
        'axes.titleweight': 'bold',
        'axes.titleweight': 'bold'
    })

## Chargement des données

Description des données via [Kernel Kaggle](https://www.kaggle.com/code/willkoehrsen/start-here-a-gentle-introduction/notebook): <br>

   - application_train/application_test: the main training and testing data with information about each loan application at Home Credit. Every loan has its own row and is identified by the feature SK_ID_CURR. The training application data comes with the TARGET indicating 0: the loan was repaid or 1: the loan was not repaid.
   - bureau: data concerning client's previous credits from other financial institutions. Each previous credit has its own row in bureau, but one loan in the application data can have multiple previous credits.
   - bureau_balance: monthly data about the previous credits in bureau. Each row is one month of a previous credit, and a single previous credit can have multiple rows, one for each month of the credit length.
   - previous_application: previous applications for loans at Home Credit of clients who have loans in the application data. Each current loan in the application data can have multiple previous loans. Each previous application has one row and is identified by the feature SK_ID_PREV.
   - POS_CASH_BALANCE: monthly data about previous point of sale or cash loans clients have had with Home Credit. Each row is one month of a previous point of sale or cash loan, and a single previous loan can have many rows.
   - credit_card_balance: monthly data about previous credit cards clients have had with Home Credit. Each row is one month of a credit card balance, and a single credit card can have many rows.
    installments_payment: payment history for previous loans at Home Credit. There is one row for every made payment and one row for every missed payment.


### Schéma des données 

![schema_dataset](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

### Lectures des données

In [ ]:
# Path to the dataset folder
dataset_path = 'dataset'

# List all files in the folder
files = os.listdir(dataset_path)

# Filter for CSV files
csv_files = [file for file in files if file.endswith('.csv')]

print(
    f"Fichier CSV de notre jeu de donnée ({len(files)} fichiers):  \n {files}")

In [2]:
# Create Dataframes with CSV from dataset

app_train = pd.read_csv("dataset/application_train.csv")
app_test = pd.read_csv("dataset/application_test.csv")
bureau = pd.read_csv("dataset/bureau.csv")
bureau_balance = pd.read_csv("dataset/bureau_balance.csv")
credit_card_balance = pd.read_csv("dataset/credit_card_balance.csv")
installments_payments = pd.read_csv("dataset/installments_payments.csv")
pos_cash_balance = pd.read_csv("dataset/POS_CASH_balance.csv")
previous_application = pd.read_csv("dataset/previous_application.csv")
sample_submission = pd.read_csv("dataset/sample_submission.csv")

In [ ]:
# Get description csv of each dataset
pd.set_option("max_colwidth", 400)
description = pd.read_csv(
    "dataset/HomeCredit_columns_description.csv", encoding="ISO-8859-1", index_col=0)

In [ ]:
display(description)

### Résumer des informations du dataset

In [ ]:
def summarize_datasets(folder_path):
    summary = []
    for filename in os.listdir(folder_path):
        if filename == 'HomeCredit_columns_description.csv':
            continue  # Skip the excluded file

        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)

        # Calculate summary statistics
        num_rows = df.shape[0]
        num_cols = df.shape[1]
        percent_missing = df.isnull().mean().mean() * 100
        percent_duplicates = df.duplicated().mean() * 100
        dtype_counts = df.dtypes.value_counts()
        summary.append({
            'Filename': filename,
            'Num_Rows': num_rows,
            'Num_Cols': num_cols,
            'Percent_Missing': percent_missing,
            'Percent_Duplicates': percent_duplicates,
            'Object_Type_Count': dtype_counts.get('object', 0),
            'Float_Type_Count': dtype_counts.get('float64', 0),
            'Int_Type_Count': dtype_counts.get('int64', 0),
            'Bool_Type_Count': dtype_counts.get('bool', 0)
        })

    summary_df = pd.DataFrame(summary)
    return summary_df


# Use function
folder_path = 'dataset'
dataset_summary = summarize_datasets(folder_path)
display(dataset_summary)

## Analyse Exploration des données: application_train.csv | test.csv

La première partie de l'exploration de nos données va se concentrer sur le jeu de donnée application_train.csv & test.csv,pour appronfondir l'analyse des features de ces principaux fichiers.

In [ ]:
# Get shape of dataframes
print('Nombre de features et ligne dans le dataframe app_train:', app_train.shape)
print('Nombre de features et ligne dans le dataframe app_test:', app_test.shape)

On remarque de que nos données app_train contient beaucoup plus de données que app_test,avec **307511 observations** et **122 features**. On remarque aussi que une colonne est manquante par rapport à app_train. Nous allons vérifier quelle colonne diffère entre ces deux dataframes.

In [ ]:
# Check if 'TARGET' is the only difference
print("Vérification de la colonne manquantes entre les deux dataframes")
display(app_train.columns.difference(app_test.columns))

La colonne manquantes dans app_test est la valeur TARGET. C'est cette valeur que l'on veut prédire.

In [ ]:
# Distribution of the Target column
app_train['TARGET'].value_counts()

In [ ]:
ax, fig = plt.subplots(figsize=(20, 8))
ax = sns.countplot(y='TARGET', data=app_train, palette=["darkblue", "darkred"])
ax.set_title("TARGET distribution")

for p in ax.patches:
    percentage = '{:.1f}%'.format(
        100 * p.get_width()/len(app_train.TARGET))
    x = p.get_x() + p.get_width()
    y = p.get_y() + p.get_height()/2
    ax.annotate(percentage, (x, y), fontsize=20, fontweight='bold')

On remarque grâce a ce graphique une distribution de la colonne 'TARGET' très déséquilibrer,on a beaucoup de prêts remboursés que de prêt non-remboursés.Ce problème d'imbalance des classes peut réduire la performance de notre algorithmes d'apprentissage.

### Analyse des valeurs manquantes

In [ ]:
# Create a function to see the completion of dataframe per column
def missing_value_dataframe(dataframe: pd.DataFrame):
    """Function for calculating missing values per column to dataframe. 
    Get the percentage of non-missing and total missing values per column

    parameters :
        dataframe : pandas dataframe to calculate missing values
    return :
    Specific dataframe with missing values per column   
    """
    # Count missing values per column
    missing_count = dataframe.isnull().sum()

    # Count non-missing values per column
    non_missing_count = dataframe.notnull().sum()

    # Calculate the total number of values per column
    total_count = len(dataframe)

    # Calculate the percentage of non-missing values
    percentage_non_missing = (non_missing_count / total_count) * 100

    # Create a new DataFrame with the values
    df_missing_values = pd.DataFrame({
        'Taux de remplissage': percentage_non_missing,
        'Nombre de valeurs manquantes': missing_count})
    df_missing_values = df_missing_values.sort_values(
        by='Taux de remplissage', ascending=False)

    return df_missing_values

In [ ]:
# Aplly function to dataframe
df_missing_values_app_train = missing_value_dataframe(app_train)

filtered_df = df_missing_values_app_train[df_missing_values_app_train['Taux de remplissage'] < 100]
display(filtered_df)

In [ ]:
# Calculate the count of missing values per column
missing_counts = app_train.isnull().sum().sort_values(ascending=False)

# Select columns with the most null value
top_10_missing = missing_counts.head(50).index

# Fitered the dataframe to keep top 10 missing columns
df_filtered_10_missing = app_train[top_10_missing]

# Use missingno.matrix() on a DataFrame filtered to include only these columns
msno.matrix(df_filtered_10_missing)

Certaines modèles telle-que XGboost peuvent gérer les valeurs manquantes sans impution. Nous avons plusieurs choix supprimer les colonnes avec trop de valeurs manquantes, les remplacer avec une méthodes d'imputation. Pour l'instant nous pouvons pas déterminer si nous allons garder ou non ces colonnes étant peu avancer dans l'analyse.
Nous pouvons remarquer qu'un certain pattern ce dessine dans les valeurs manquantes et les valeurs sont les mêmes bien souvent que dans d'autres colonnes. On remarque aussi que les colonnes avec le plus de Nan correspondent à des caractéristique sur l'habitat et non sur les crédits.

### Types des colonnes 

Regardons maintenant le nombre de types de colonnes différentes que nous avons, les variables INT et FLOAT sont des variables numérique et 'object' contient des string et sont des variables catégorielles.

In [ ]:
# Number of each type of column
app_train.dtypes.value_counts()

Regardons maintenant les colonnes 'object' (variables catégorielles) 

In [ ]:
# Number of unique classes in each object column
app_train.select_dtypes('object').apply(pd.Series.nunique, axis=0)

La plupart des variables catégorielles ont un nombre relativement petit d'entrées uniques. Nous devrons trouver un moyen de traiter ces variables catégorielles…

### Anomalies dans notre jeu de donnée 

In [ ]:
app_train['DAYS_BIRTH'].describe()

On remarque que notre colonne 'DAYS_BIRTH' contient des valeurs négative car ils sont enregistrés par rapport à la demande de prêt en cours. Il est nécessaire de modifier cette variable pour obtenir des chiffres plus compréhensibles pour l'analyse.

In [ ]:
(app_train['DAYS_BIRTH'] / -365).describe()

In [ ]:
# Test if function was apply
(app_train['DAYS_BIRTH']).describe()

In [ ]:
fig = px.histogram(app_train, x='DAYS_BIRTH', nbins=30,
                   title="Représentation de l'Age des clients")
fig.show()

In [ ]:
app_train['DAYS_EMPLOYED'].describe()

In [ ]:
fig = px.histogram(app_train, x='DAYS_EMPLOYED', nbins=30,
                   title="Représentation des jours d'emploi")
fig.show()

Idem ces données ne sont pas "normales" au sens où le Max. représente 1000 années (365243/365j). Est-ce un individu isolé? Plusieurs individus de l'échantillon?

In [ ]:
print("%0.0f values with 365243 days employed for training data" %
      len(app_train[app_train['DAYS_EMPLOYED'] == 365243]))
print("%0.0f Total values from days employed for training data" %
      app_train.shape[0])
print("***********************")
print("%0.0f values with 365243 days employed for testing data" %
      len(app_test[app_test['DAYS_EMPLOYED'] == 365243]))
print("%0.0f Total values from days employed for testing data" %
      app_test.shape[0])

In [ ]:
# Create an outliers flag column
app_train['DAYS_EMPLOYED_OUTLIERS'] = app_train["DAYS_EMPLOYED"] == 365243
app_test['DAYS_EMPLOYED_OUTLIERS'] = app_test["DAYS_EMPLOYED"] == 365243

# Replace outliers values with nan
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace=True)

In [ ]:
# Create columns with DAYS_EMPLOYEMENT anomalies in app_test
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace=True)
print('There are %d anomalies in the test data out of %d entries' %
      (app_test["DAYS_EMPLOYED_ANOM"].sum(), len(app_test)))

In [ ]:
fig = px.histogram(app_train, x='DAYS_EMPLOYED', nbins=30,
                   title="Représentation des jours d'emploi après remplacement des outliers")
fig.show()

### Analyse des variables Principales

Ici nous allons les principales features qui peuvent être interessante à analyser. Nous allons analyser certaines features en univarié pour voir les liens avec notre colonne TARGET.Nous voir l'impact de certaines features sur le nom remboursement d'un prêt.

In [ ]:


# Mise à jour de la fonction pour calculer correctement les pourcentages


def calculate_percentages(total_count, subset_count):
    percentage = (subset_count / total_count) * 100
    return f"{percentage:.2f}%"


# Création de subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=(
    "Tout les contrats", "Taux de non-remboursement en fonction du type de contrats"))

# Calcul du nombre total et des pourcentages pour chaque type de contrat
total_cash_loans = app_train[app_train['NAME_CONTRACT_TYPE']
                             == 'Cash loans'].shape[0]
total_revolving_loans = app_train[app_train['NAME_CONTRACT_TYPE']
                                  == 'Revolving loans'].shape[0]

# Pour le premier plot
cash_loans_percent = calculate_percentages(
    app_train.shape[0], total_cash_loans)
revolving_loans_percent = calculate_percentages(
    app_train.shape[0], total_revolving_loans)

# Ajout des traces pour le premier plot
fig.add_trace(
    go.Bar(y=['Cash loans', 'Revolving loans'], x=[total_cash_loans, total_revolving_loans], orientation='h',
           marker=dict(color=['blue', 'red']), text=[cash_loans_percent, revolving_loans_percent]),
    row=1, col=1
)

# Calcul des nombres et pourcentages pour les contrats avec TARGET == 1
cash_loans_target_1 = app_train[(app_train['NAME_CONTRACT_TYPE'] == 'Cash loans') & (
    app_train['TARGET'] == 1)].shape[0]
revolving_loans_target_1 = app_train[(
    app_train['NAME_CONTRACT_TYPE'] == 'Revolving loans') & (app_train['TARGET'] == 1)].shape[0]

cash_loans_percent_target_1 = calculate_percentages(
    total_cash_loans, cash_loans_target_1)
revolving_loans_percent_target_1 = calculate_percentages(
    total_revolving_loans, revolving_loans_target_1)

# Ajout des traces pour le deuxième plot
fig.add_trace(
    go.Bar(y=['Cash loans', 'Revolving loans'], x=[cash_loans_target_1, revolving_loans_target_1], orientation='h',
           marker=dict(color=['blue', 'red']), text=[cash_loans_percent_target_1, revolving_loans_percent_target_1]),
    row=1, col=2
)

# Mise à jour de la mise en page et affichage du plot
fig.update_layout(height=600, width=800,
                  title_text="Représentation des types de contrats", showlegend=False)
fig.show()

Sur le premier graphique on remarque que les prêt renouvelable ne forme qu'une petit des prêts accordés (10%).Par contre il représente un grand nombre des prêts qui sont non-remboursés en proportion du nombre de prêts renouvelable accordés.
**1 prêt renouvelable sur 2 n'est pas remboursés.**

In [ ]:
# Function to correctly calculate percentages
def calculate_percentages(total_count, subset_count):
    percentage = (subset_count / total_count) * 100
    return f"{percentage:.2f}%"


# Creating subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=(
    "Contrats attribués en fonction du genre", "Taux de non-remboursement en fonction du genre"))

# Calculating total counts and percentages for each gender category
total_male = app_train[app_train['CODE_GENDER'] == 'M'].shape[0]
total_female = app_train[app_train['CODE_GENDER'] == 'F'].shape[0]

# For the first plot
male_percent = calculate_percentages(app_train.shape[0], total_male)
female_percent = calculate_percentages(app_train.shape[0], total_female)

# Adding traces for the first plot
fig.add_trace(
    go.Bar(y=['Male', 'Female'], x=[total_male, total_female], orientation='h',
           marker=dict(color=['green', 'purple']), text=[male_percent, female_percent]),
    row=1, col=1
)

# Calculating counts and percentages for gender categories with TARGET == 1
male_target_1 = app_train[(app_train['CODE_GENDER'] == 'M') & (
    app_train['TARGET'] == 1)].shape[0]
female_target_1 = app_train[(app_train['CODE_GENDER'] == 'F') & (
    app_train['TARGET'] == 1)].shape[0]

male_percent_target_1 = calculate_percentages(
    app_train.shape[0], male_target_1)
female_percent_target_1 = calculate_percentages(
    app_train.shape[0], female_target_1)

# Adding traces for the second plot
fig.add_trace(
    go.Bar(y=['Male', 'Female'], x=[male_target_1, female_target_1], orientation='h',
           marker=dict(color=['green', 'purple']), text=[male_percent_target_1, female_percent_target_1]),
    row=1, col=2
)

# Updating the layout and displaying the plot
fig.update_layout(height=600, width=1000,
                  title_text="Représentation des prêts en fonction du Genre", showlegend=False)
fig.show()

Sur ce graphique on remarque les clients sont le double des clients masculin. En porportion les hommes ont plus de chances de ne pas remboursés leurs prêts.

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Function to correctly calculate percentages for subsets


def calculate_percentages(total_count, subset_count):
    percentage = (subset_count / total_count) * 100
    return f"{percentage:.2f}%"


# Creating subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=(
    "Prêts accordés propriétaire Voiture", "Taux de non-remboursement propriétaire voiture"))

# Calculating total counts and percentages for car ownership categories
total_own_car = app_train[app_train['FLAG_OWN_CAR'] == 'Y'].shape[0]
total_no_car = app_train[app_train['FLAG_OWN_CAR'] == 'N'].shape[0]

# For the first plot
own_car_percent = calculate_percentages(app_train.shape[0], total_own_car)
no_car_percent = calculate_percentages(app_train.shape[0], total_no_car)

# Adding traces for the first plot
fig.add_trace(
    go.Bar(y=['Propriétaire de Voiture', 'Non propriétaire'], x=[total_own_car, total_no_car], orientation='h',
           marker=dict(color=['green', 'gray']), text=[own_car_percent, no_car_percent]),
    row=1, col=1
)

# Calculating counts and percentages for car ownership categories with TARGET == 1
own_car_target_1 = app_train[(app_train['FLAG_OWN_CAR'] == 'Y') & (
    app_train['TARGET'] == 1)].shape[0]
no_car_target_1 = app_train[(app_train['FLAG_OWN_CAR'] == 'N') & (
    app_train['TARGET'] == 1)].shape[0]

own_car_percent_target_1 = calculate_percentages(
    total_own_car, own_car_target_1)
no_car_percent_target_1 = calculate_percentages(total_no_car, no_car_target_1)

# Adding traces for the second plot
fig.add_trace(
    go.Bar(y=['Propriétaire de Voiture', 'Non propriétaire'], x=[own_car_target_1, no_car_target_1], orientation='h',
           marker=dict(color=['green', 'gray']), text=[own_car_percent_target_1, no_car_percent_target_1]),
    row=1, col=2
)

# Updating the layout and displaying the plot
fig.update_layout(height=600, width=1000,
                  title_text="Répresentation des prêts en fonction des propriétaires de Voiture ", showlegend=False)
fig.show()

Les deux catégories (propriétaire ou non) ont des taux de non-remboursement d'environ 8%.

In [ ]:
# Function to correctly calculate percentages for subsets
def calculate_percentages(total_count, subset_count):
    percentage = (subset_count / total_count) * 100
    return f"{percentage:.2f}%"


# Creating subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=(
    "Prêts accordés par Statut familial", "Taux de non-remboursement par Statut familial"))

# Define a list of colors
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']

# List of family statuses
family_statuses = ['Single / not married', 'Married',
                   'Civil marriage', 'Widow', 'Separated', 'Unknown']

# Calculating total counts for each family status
status_counts = [(status, app_train[app_train['NAME_FAMILY_STATUS']
                  == status].shape[0]) for status in family_statuses]

# Sorting by counts
status_counts_sorted = sorted(status_counts, key=lambda x: x[1])

# Splitting the tuples for plotting
statuses_sorted, counts_sorted = zip(*status_counts_sorted)

# Calculating percentages
percentages_sorted = [calculate_percentages(
    app_train.shape[0], count) for count in counts_sorted]

# Adding traces for the first plot, sorted by counts, with colors
fig.add_trace(
    go.Bar(y=list(statuses_sorted), x=list(counts_sorted), orientation='h',
           marker=dict(color=colors), text=percentages_sorted),
    row=1, col=1
)

# Calculating counts and percentages for family status categories with TARGET == 1, and sorting
counts_target_1_sorted = [app_train[(app_train['NAME_FAMILY_STATUS'] == status) & (
    app_train['TARGET'] == 1)].shape[0] for status in statuses_sorted]
percentages_target_1_sorted = [calculate_percentages(
    counts_sorted[i], count_target_1) for i, count_target_1 in enumerate(counts_target_1_sorted)]

# Adding traces for the second plot, sorted by counts with TARGET == 1, with colors
fig.add_trace(
    go.Bar(y=list(statuses_sorted), x=counts_target_1_sorted, orientation='h',
           marker=dict(color=colors), text=percentages_target_1_sorted),
    row=1, col=2
)

# Updating the layout and displaying the plot
fig.update_layout(height=600, width=1000,
                  title_text="Répresentation des prêts en fonction du Statut familial", showlegend=False)
fig.show()

La plupart des clients sont mariés, suivis des célibataires / non mariés et des mariages civils.

En termes de pourcentage de non-remboursement du prêt, le mariage civil a le pourcentage le plus élevé de non-remboursement (10%), la veuve étant le plus bas (à l'exception de l'inconnu).

In [ ]:
# Assuming 'app_train' is your DataFrame

# Define a list of colors for the different income types
colors = ['#17becf', '#bcbd22', '#7f7f7f', '#e377c2',
          '#8c564b', '#9467bd', '#d62728', '#2ca02c']

# List of income types
income_types = ['Working', 'State servant', 'Commercial associate',
                'Pensioner', 'Unemployed', 'Student', 'Businessman', 'Maternity leave']

# Calculating total counts for each income type
income_counts = [(income, app_train[app_train['NAME_INCOME_TYPE']
                  == income].shape[0]) for income in income_types]

# Sorting by counts
income_counts_sorted = sorted(income_counts, key=lambda x: x[1])

# Splitting the tuples for plotting
incomes_sorted, counts_sorted = zip(*income_counts_sorted)

# Calculating percentages
percentages_sorted = [calculate_percentages(
    app_train.shape[0], count) for count in counts_sorted]

# Creating subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=(
    "Prêts accordés par Type de revenu", "Taux de non-remboursement par Type de revenu"))

# Adding traces for the first plot, sorted by counts, with colors
fig.add_trace(
    go.Bar(y=list(incomes_sorted), x=list(counts_sorted), orientation='h',
           marker=dict(color=colors), text=percentages_sorted),
    row=1, col=1
)

# Calculating counts and percentages for income types with TARGET == 1, and sorting
counts_target_1_sorted = [app_train[(app_train['NAME_INCOME_TYPE'] == income) & (
    app_train['TARGET'] == 1)].shape[0] for income in incomes_sorted]
percentages_target_1_sorted = [calculate_percentages(
    counts_sorted[i], count_target_1) for i, count_target_1 in enumerate(counts_target_1_sorted)]
print(counts_target_1_sorted)
# Adding traces for the second plot, sorted by counts with TARGET == 1, with colors
fig.add_trace(
    go.Bar(y=list(incomes_sorted), x=counts_target_1_sorted, orientation='h',
           marker=dict(color=colors), text=percentages_target_1_sorted),
    row=1, col=2
)

# Updating the layout and displaying the plot
fig.update_layout(height=600, width=1000,
                  title_text="Répresentation des prêts en fonction du Type de revenu", showlegend=False)
fig.show()



La plupart des demandeurs de prêts sont des revenus du travail, suivis par un associé commercial, un retraité et un fonctionnaire.

Les demandeurs avec le type de revenu Congé de maternité ont un ratio de près de 40% de prêts non remboursés, suivis des chômeurs (37%). Les autres types de revenus sont inférieurs à la moyenne de 10% pour ne pas rembourser les prêts.


In [ ]:
# Liste des types d'occupation
occupation_types = [
    'Laborers', 'Core staff', 'Accountants', 'Managers',
    'Drivers', 'Sales staff', 'Cleaning staff', 'Cooking staff',
    'Private service staff', 'Medicine staff', 'Security staff',
    'High skill tech staff', 'Waiters/barmen staff',
    'Low-skill Laborers', 'Realty agents', 'Secretaries', 'IT staff',
    'HR staff'
]

# Calcul des totaux pour chaque type d'occupation
occupation_counts = [(occupation, app_train[app_train['OCCUPATION_TYPE']
                      == occupation].shape[0]) for occupation in occupation_types]

# Tri par les totaux
occupation_counts_sorted_by_total = sorted(
    occupation_counts, key=lambda x: x[1])

# Séparation des tuples pour le traçage, triés par total
occupations_sorted_by_total, counts_sorted_by_total = zip(
    *occupation_counts_sorted_by_total)

# Calcul des pourcentages
percentages_sorted_by_total = [calculate_percentages(
    app_train.shape[0], count) for count in counts_sorted_by_total]

# Calcul des totaux et des pourcentages pour les cas avec TARGET == 1, et tri
occupation_counts_with_target = [(occupation, app_train[(app_train['OCCUPATION_TYPE'] == occupation) & (
    app_train['TARGET'] == 1)].shape[0]) for occupation in occupation_types]
occupation_counts_sorted_by_target = sorted(
    occupation_counts_with_target, key=lambda x: x[1], reverse=False)

# Séparation des tuples pour le traçage, triés par TARGET == 1
occupations_sorted_by_target, counts_sorted_by_target = zip(
    *occupation_counts_sorted_by_target)
percentages_sorted_by_target = [calculate_percentages(
    count, app_train[app_train['OCCUPATION_TYPE'] == occupation].shape[0]) for occupation, count in occupation_counts_sorted_by_target]

# Création de subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=(
    "Prêts accordés par Type d'occupation", "Taux de non-remboursement par Type d'occupation"))

# Ajout des traces pour le premier plot, trié par les totaux
fig.add_trace(
    go.Bar(y=list(occupations_sorted_by_total), x=list(counts_sorted_by_total), orientation='h',
           marker=dict(color=colors), text=percentages_sorted_by_total),
    row=1, col=1
)

# Ajout des traces pour le second plot, trié par TARGET == 1
fig.add_trace(
    go.Bar(y=list(occupations_sorted_by_target), x=list(counts_sorted_by_target), orientation='h',
           marker=dict(color=colors), text=percentages_sorted_by_target),
    row=1, col=2
)

# Mise à jour de la disposition et affichage du graphique
fig.update_layout(height=600, width=1000,
                  title_text="Représentation des prêts en fonction du Type d'occupation", showlegend=False)
fig.show()

La plupart des prêts sont contractés par des ouvriers, suivis par les vendeurs/commerciaux. Le personnel informatique prend le montant de prêts le plus bas.

La catégorie avec le pourcentage le plus élevé de prêts non remboursés est celle des ouvriers peu qualifiés (plus de 17%), suivis des chauffeurs et des serveurs / barmen, du personnel de sécurité, des ouvriers et du personnel de cuisine.

In [ ]:
# Define a list of colors for the different education types
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

# List of education types
education_types = [
    'Secondary / secondary special',
    'Higher education',
    'Incomplete higher',
    'Lower secondary',
    'Academic degree'
]

# Calculating total counts and counts with TARGET == 1 for each education type
education_data = []
for education in education_types:
    total = app_train[app_train['NAME_EDUCATION_TYPE'] == education].shape[0]
    defaults = app_train[(app_train['NAME_EDUCATION_TYPE'] == education) & (
        app_train['TARGET'] == 1)].shape[0]
    default_rate = (defaults / total) * 100 if total > 0 else 0
    education_data.append((education, total, defaults, default_rate))

# Sorting by total counts for the first plot
education_data_sorted_by_total = sorted(education_data, key=lambda x: x[1])

# Sorting by default rate for the second plot
education_data_sorted_by_default_rate = sorted(
    education_data, key=lambda x: x[3], reverse=False)

# Splitting the data for plotting
educations_sorted_by_total, _, _, _ = zip(*education_data_sorted_by_total)
_, counts_sorted_by_total, _, percentages_sorted_by_total = zip(
    *education_data_sorted_by_total)

educations_sorted_by_default_rate, _, counts_sorted_by_default_rate, percentages_sorted_by_default_rate = zip(
    *education_data_sorted_by_default_rate)

# Creating subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=(
    "Prêts accordés par Type d'éducation", "Taux de non-remboursement par Type d'éducation"))

# Adding traces for the first plot
fig.add_trace(
    go.Bar(y=list(educations_sorted_by_total), x=list(counts_sorted_by_total), orientation='h',
           marker=dict(color=colors), text=[f"{p:.2f}%" for p in percentages_sorted_by_total]),
    row=1, col=1
)

# Adding traces for the second plot, with a different order based on default rate
fig.add_trace(
    go.Bar(y=list(educations_sorted_by_default_rate), x=list(percentages_sorted_by_default_rate), orientation='h',
           marker=dict(color=colors), text=[f"{c} ({p:.2f}%)" for c, p in zip(counts_sorted_by_default_rate, percentages_sorted_by_default_rate)]),
    row=1, col=2
)

# Updating the layout and displaying the plot
fig.update_layout(height=600, width=1000,
                  title_text="Représentation des prêts en fonction du Type d'éducation", showlegend=False)
fig.show()

La majorité des clients ont une éducation dans l'éducation secondaire, suivis des clients avec une éducation supérieure. Un très petit nombre d'emprunteur possède un diplôme universitaire.

La catégorie du premier cycle du secondaire, bien que rare, a le taux le plus élevé de non-remboursement du prêt (11%). Les personnes ayant un diplôme universitaire ont un taux de non-remboursement inférieur à 2%.

In [ ]:
# Génération d'une palette de couleurs
colors = ['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a', '#d62728', '#ff9896',
          '#9467bd', '#c5b0d5', '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7',
          '#bcbd22', '#dbdb8d', '#17becf', '#9edae5']

# Liste des types de logement
housing_types = [
    'House / apartment', 'Rented apartment', 'With parents',
    'Municipal apartment', 'Office apartment', 'Co-op apartment'
]

# Calcul des totaux et des pourcentages pour les cas avec TARGET == 1
housing_data = []
for housing in housing_types:
    total_count = app_train[app_train['NAME_HOUSING_TYPE'] == housing].shape[0]
    target_count = app_train[(app_train['NAME_HOUSING_TYPE'] == housing) & (
        app_train['TARGET'] == 1)].shape[0]
    percentage = (target_count / total_count) * 100 if total_count > 0 else 0
    housing_data.append((housing, total_count, target_count, percentage))

# Tri par le total pour le premier subplot
housing_sorted_by_total = sorted(
    housing_data, key=lambda x: x[1], reverse=False)

# Tri par le pourcentage pour le second subplot
housing_sorted_by_percentage = sorted(
    housing_data, key=lambda x: x[3], reverse=False)

# Création des subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=(
    "Prêts accordés par Type de logement", "Taux de non-remboursement par Type de logement"))

# Ajout des traces pour le premier subplot
for i, (housing, total_count, _, _) in enumerate(housing_sorted_by_total):
    fig.add_trace(go.Bar(y=[housing], x=[total_count], orientation='h', marker=dict(color=colors[i % len(colors)])),
                  row=1, col=1)

# Ajout des traces pour le second subplot
for i, (housing, _, _, percentage) in enumerate(housing_sorted_by_percentage):
    fig.add_trace(go.Bar(y=[housing], x=[percentage], orientation='h', marker=dict(color=colors[i % len(colors)]), text=f"{percentage:.2f}%"),
                  row=1, col=2)

# Mise à jour de la disposition et affichage du graphique
fig.update_layout(height=600, width=1000, showlegend=False,
                  title_text="Représentation des prêts en fonction du Type de logement")
fig.show()

Plus de 250 000 demandeurs de crédits vivent en maison ou appartement. Les catégories suivantes, faible pourcentage, représentent une population moins "indépendante" (vivre chez ses parents, etc…).

Dans ces catégories, les loueurs d'appartements (non propriétaires de leur résidence principale), ainsi que ceux qui vivent chez leurs parents, ont un taux de non-remboursement supérieur à 10%.

### Correlations 

Maintenant que nous avons traiter les outliers et les variables catégorielle. Nous allons voir les corrélations qu'il existe entre nos colonnes(Features) et notre TARGET pour évaluer les liens entre nos variables et choisir a features les plus pertinantes. Pour faire cela nous allons calculer le coefficient de corrélation de Person. 

In [ ]:
# Find correlations with the target and sort
correlations = app_train.corr(numeric_only=True)['TARGET'].sort_values()

# Display correlations
print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

Notons que la corrélation la plus forte est celle de la variable 'DAYS_BIRTH'. Ceci étant, pour faciliter la compréhension est retrouver la logique vue en préambule, les jours exprimés en valeurs négatives peuvent être traités en valeurs absolues. Alors le coef. de Person sera négatif, cela expose le fait qu'un client plus âgé sera moins susceptible de faire défaut au remboursement de son crédit (cela peut sembler logique, en tout cas ce n'est pas absurde).

In [ ]:
# Find the correlation of the positive days since birth and target
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

En termes de distribution des âges elle peut uniquement servir à visualiser la non présence d'outliers, suite à ce qui a été fait en amont sur les valeurs négatives de départ. Pour visualiser l'effet de l'âge sur la Target, nous pouvons faire un graphique (KDE) coloré par la valeur TARGET 0 et 1.

In [ ]:
# Calcul de l'âge en années
ages_years = app_train['DAYS_BIRTH'] / 365

# Création du graphique
fig = go.Figure()

# Ajout d'un histogramme à la figure
fig.add_trace(go.Histogram(
    x=ages_years, nbinsx=50,
    marker=dict(line=dict(color='black', width=1))
))

# Mise à jour du layout pour ajouter les titres et personnaliser l'apparence
fig.update_layout(
    title_text='Age des clients',
    xaxis_title_text='Age (Année)',
    yaxis_title_text='Nombres',
    bargap=0.2,  # Espace entre les barres de l'histogramme
    template='plotly_white'  # Utilise un fond blanc pour le style
)

# Affichage du graphique
fig.show()

In [ ]:

# Calcul de l'âge en années
age_paid = app_train.loc[app_train['TARGET'] == 0, 'DAYS_BIRTH'] / 365
age_not_paid = app_train.loc[app_train['TARGET'] == 1, 'DAYS_BIRTH'] / 365

# Création d'une liste de données
hist_data = [age_paid, age_not_paid]

# Étiquettes pour les groupes de données
group_labels = ['target == 0', 'target == 1']

# Création du KDE plot
fig = ff.create_distplot(hist_data, group_labels,
                         show_hist=False, show_rug=False)

# Ajout du titre et des étiquettes des axes
fig.update_layout(title_text='Distribution des Ages',
                  xaxis_title='Age (Année)',
                  yaxis_title='Densité')

# Affichage du graphique
fig.show()

La courbe cible TARGET 1 s'incline vers l'extrémité la plus jeune de la plage. Bien qu'il ne s'agisse pas d'une corrélation significative (coefficient Pearson -0,07), cette variable sera probablement utile dans un modèle d'apprentissage car elle affecte la Target.
essayons maintenant de regrouper les âges pour avoir un graphique plus précis sur taux de non-recouvrement des prêts en fonction de l'âge. Pour cela nous allons regrouper les âges présent par tranche de 5 ans.

In [ ]:
# Age information into a separate dataframe
age_data = app_train[['TARGET', 'DAYS_BIRTH']]
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH'] / 365

# Bin the age data
age_data['YEARS_BINNED'] = pd.cut(
    age_data['YEARS_BIRTH'], bins=np.linspace(20, 70, num=11))
age_data.head(10)

In [ ]:
# Group by the bin and calculate averages
age_groups = age_data.groupby('YEARS_BINNED').mean()
age_groups

In [ ]:
# Création du graphique de barres
fig = go.Figure()

# Ajout des barres pour chaque groupe d'âge
fig.add_trace(go.Bar(
    # Conversion de l'index en chaîne de caractères pour l'affichage
    x=age_groups.index.astype(str),
    # Calcul du pourcentage d'échec de remboursement
    y=100 * age_groups['TARGET'],
    marker=dict(line=dict(color='black', width=1))
))

# Mise à jour du layout pour ajouter les titres et personnaliser l'apparence
fig.update_layout(
    title_text="Non-remboursement d'un prêt par groupe d'âge",
    xaxis_title='Groupe âge (Années)',
    yaxis_title='Taux de non-remboursement (%)',
    xaxis_tickangle=-45,  # Rotation des étiquettes sur l'axe des x
    template='plotly_white'  # Utilisation d'un fond blanc pour le style
)

# Affichage du graphique
fig.show()

On vois que la tendance est bien réelle et décroissante en fonction de l'âge, le groupe d'âge des 20-25 ans on 12% de chance de ne pas rembourser leurs prêts contre 4% pour les 65-70 ans.

### Sources extérieurs

Ces 3 variables (EXT_SOURCE) présentant les corrélations négatives les plus fortes avec la Target. Selon la documentation, ces fonctionnalités représentent un «score normalisé à partir d'une source de données externe». Difficile de comprendre le sens exact, nous pouvons émettre l'hypothèse d'une cote de crédit cumulative établie à l'aide de différentes sources de données.

In [ ]:
# Calcul de la matrice de corrélations pour les variables spécifiées
corr_matrix = app_train[[
    'TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']].corr()

# Création du graphique de heatmap
fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,  # Valeurs de corrélation
    x=corr_matrix.columns,  # Noms des colonnes pour l'axe des x
    y=corr_matrix.index,  # Noms des lignes pour l'axe des y
    colorscale='RdBu_r',  # Palette de couleurs
    zmin=-0.25,  # Minimum de l'échelle de couleurs
    zmax=0.6,  # Maximum de l'échelle de couleurs
))

# Ajout des annotations pour chaque cellule
for i, row in enumerate(corr_matrix.values):
    for j, value in enumerate(row):
        fig.add_annotation(x=corr_matrix.columns[j], y=corr_matrix.index[i],
                           text=str(round(value, 2)),
                           showarrow=False, font=dict(color="black"))

# Mise à jour des titres et du layout
fig.update_layout(
    title_text='EXT_SOURCE Correlation Heatmap',
    title_x=0.5,  # Centrer le titre
    # Positionnement des labels de l'axe des x en haut
    xaxis=dict(tickmode='array', side='top'),
    # Inversion de l'axe des y pour correspondre à l'ordre habituel des heatmaps
    yaxis=dict(tickmode='array', autorange='reversed'),
    template='plotly_white'
)

# Affichage de la heatmap
fig.show()

Ces sources extérieures sont corrélées négativement avec la Target (Attention, négativement mais avec de très faibles coefficients). Malgré tout, on peut penser que si la valeur augmente, le client est plus susceptible de pouvoir rembourser le crédit. Notons que 'DAYS_BIRTH' est corrélée avec 'EXT_SOURCE_1', peut-être l'un des facteurs de ce score est l'âge du client.<br>
Visualisons maintenant la distribution de ces feature en fonction de TARGET.

In [ ]:
# Prepare subplot figure with 3 rows
fig = make_subplots(rows=3, cols=1, subplot_titles=['Distribution de EXT_SOURCE_1 par Valeur de TARGET',
                                                    'Distribution de EXT_SOURCE_2 par Valeur de TARGET',
                                                    'Distribution de EXT_SOURCE_3 par Valeur de TARGET'])

# Iterate through the sources
for i, source in enumerate(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']):
    # Data for repaid loans and loans that were not repaid
    data_repaid = app_train.loc[app_train['TARGET'] == 0, source].dropna()
    data_not_repaid = app_train.loc[app_train['TARGET'] == 1, source].dropna()

    # Create distplot for each source
    # Note: ff.create_distplot returns a figure object which we need to deconstruct to use in subplots
    fig_repaid = ff.create_distplot(
        [data_repaid], ['Repaid'], show_hist=False, show_rug=False)
    fig_not_repaid = ff.create_distplot(
        [data_not_repaid], ['Not Repaid'], show_hist=False, show_rug=False)

    # Adding traces from created distplots to the main subplot figure
    for trace in fig_repaid['data']:
        fig.add_trace(go.Scatter(
            trace, line=dict(color='blue')), row=i+1, col=1)
    for trace in fig_not_repaid['data']:
        fig.add_trace(go.Scatter(
            trace, line=dict(color='red')), row=i+1, col=1)

# Update layout of the subplot figure
fig.update_layout(height=1200, width=800, title_text="Distributions de EXT_SOURCE par Valeur de TARGET",
                  showlegend=False)

# Show figure
fig.show()

## Analyse Exploratoire des autres fichiers CSV


Nous allons analyser les autres fichier présent dans notre jeu de données pour pouvoir ajouter de nouvelles features pertinentes pour notre modèle.

### Analyse exploratoire fichier *bureau.csv*

Ce fichier contient tous les crédits antérieurs des clients fournis par d'autres institutions financières qui ont été déclarés au Bureau de crédit (pour les clients qui ont un prêt dans l'échantillon). Pour chaque prêt de l'échantillon, il y a autant de lignes que de nombre de crédits que le client avait au bureau de crédit avant la date de la demande.
On va fusion le dataframe app_train avec bureau pour avoir les "Target".

In [ ]:
bureau.shape

In [ ]:
application_bureau_train = app_train.merge(
    bureau, left_on='SK_ID_CURR', right_on='SK_ID_CURR', how='inner')

In [ ]:
display(application_bureau_train.head())

In [ ]:
# Génération d'une palette de couleurs
colors = ['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a', '#d62728', '#ff9896',
          '#9467bd', '#c5b0d5', '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7',
          '#bcbd22', '#dbdb8d', '#17becf', '#9edae5']

# Liste des statuts de crédit actifs
credit_active_statuses = ['Closed', 'Active', 'Sold', 'Bad debt']

# Calcul des totaux et des pourcentages pour les cas avec TARGET == 1
credit_active_data = []
for status in credit_active_statuses:
    total_count = application_bureau_train[application_bureau_train['CREDIT_ACTIVE']
                                           == status].shape[0]
    target_count = application_bureau_train[(application_bureau_train['CREDIT_ACTIVE'] == status) & (
        application_bureau_train['TARGET'] == 1)].shape[0]
    percentage = (target_count / total_count) * 100 if total_count > 0 else 0
    credit_active_data.append((status, total_count, target_count, percentage))

# Tri par le total pour le premier subplot
credit_active_sorted_by_total = sorted(
    credit_active_data, key=lambda x: x[1], reverse=False)

# Tri par le pourcentage pour le second subplot
credit_active_sorted_by_percentage = sorted(
    credit_active_data, key=lambda x: x[3], reverse=False)

# Création des subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=(
    "Prêts accordés par Statut Actif", "Taux de non-remboursement par Statut Actif"))

# Ajout des traces pour le premier subplot
for i, (status, total_count, _, _) in enumerate(credit_active_sorted_by_total):
    fig.add_trace(go.Bar(y=[status], x=[total_count], orientation='h', marker=dict(color=colors[i]),
                  text=f"{total_count}"),
                  row=1, col=1)

# Ajout des traces pour le second subplot
for i, (status, _, _, percentage) in enumerate(credit_active_sorted_by_percentage):
    fig.add_trace(go.Bar(y=[status], x=[percentage], orientation='h', marker=dict(color=colors[i]),
                  text=f"{percentage:.2f}%"),
                  row=1, col=2)

# Mise à jour de la disposition et affichage du graphique
fig.update_layout(height=600, width=1200, showlegend=False,
                  title_text="Représentation des prêts en fonction du Statut Actif")
fig.show()

### Analyse exploratoire fichier *Previous application data*

"previous_application" contient des informations sur toutes les demandes précédentes de crédit immobilier des clients qui ont des prêts dans l'échantillon. Il y a une ligne pour chaque demande précédente liée aux prêts dans notre échantillon de données. SK_ID_CURR est la clé reliant les données application_train | test aux données previous_application.

Il est nécessaire de fusionner "application_train" avec "previous_application" pour pour pouvoir extraire la part de TARGET == 1.

In [ ]:
application_prev_train = app_train.merge(
    previous_application, left_on='SK_ID_CURR', right_on='SK_ID_CURR', how='inner')

In [ ]:
# Génération d'une palette de couleurs
colors = ['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a', '#d62728', '#ff9896',
          '#9467bd', '#c5b0d5', '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7',
          '#bcbd22', '#dbdb8d', '#17becf', '#9edae5']

# Liste des types de contrat
contract_types = ['Consumer loans', 'Cash loans', 'Revolving loans', 'XNA']

# Initialisation des listes pour stocker les totaux et les pourcentages
total_counts = []
percentages_total = []
target_1_counts = []
percentages_target_1 = []

# Calcul des totaux pour chaque type de contrat
for contract in contract_types:
    total_count = application_prev_train[application_prev_train['NAME_CONTRACT_TYPE_y']
                                         == contract].shape[0]
    target_count = application_prev_train[(application_prev_train['NAME_CONTRACT_TYPE_y'] == contract) &
                                          (application_prev_train['TARGET'] == 1)].shape[0]
    total_counts.append(total_count)
    target_1_counts.append(target_count)

# Calcul des pourcentages pour les deux graphiques
total_sum = sum(total_counts)
target_1_sum = sum(target_1_counts)
percentages_total = [(count / total_sum) * 100 for count in total_counts]
percentages_target_1 = [(count / target_1_sum) *
                        100 for count in target_1_counts]

# Tri des données par ordre croissant pour les deux graphiques
sorted_indices_total = sorted(
    range(len(total_counts)), key=lambda k: total_counts[k])
sorted_indices_target_1 = sorted(
    range(len(target_1_counts)), key=lambda k: target_1_counts[k])

# Création des subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=(
    'Prêts accordés par Type de Contrat', 'Taux de non-remboursement par Type de Contrat'))

# Ajout des traces pour le premier graphique (totaux)
fig.add_trace(
    go.Bar(y=[contract_types[i] for i in sorted_indices_total],
           x=[total_counts[i] for i in sorted_indices_total],
           orientation='h',
           marker=dict(color=colors),
           text=[f"{percentages_total[i]:.2f}%" for i in sorted_indices_total]),
    row=1, col=1
)

# Ajout des traces pour le second graphique (pourcentages)
fig.add_trace(
    go.Bar(y=[contract_types[i] for i in sorted_indices_target_1],
           x=[target_1_counts[i] for i in sorted_indices_target_1],
           orientation='h',
           marker=dict(color=colors),
           text=[f"{percentages_target_1[i]:.2f}%" for i in sorted_indices_target_1]),
    row=1, col=2
)

# Mise à jour de la mise en page et affichage du graphique
fig.update_layout(height=600, width=1200, showlegend=False,
                  title_text="Représentation des prêts en fonction du Type de Contrat")
fig.show()

In [ ]:
application_prev_train['NAME_CLIENT_TYPE'].unique()

In [ ]:
def create_comparison_bar_charts(df, column_name, target_column='TARGET'):
    # Define a color palette
    colors = ['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a', '#d62728', '#ff9896',
              '#9467bd', '#c5b0d5', '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7',
              '#bcbd22', '#dbdb8d', '#17becf', '#9edae5']

    # Extract the unique values from the specified column
    unique_values = df[column_name].dropna().unique().tolist()

    # Calculate total counts and target == 1 counts
    total_counts = df[column_name].value_counts().reindex(
        unique_values).fillna(0)
    target_counts = df[df[target_column] == 1][column_name].value_counts().reindex(
        unique_values).fillna(0)
    # Calculate percentage for fist plot
    # percentage_total = (total_count)
    # Calculate percentages of non-repayment for the second plot
    percentages_target = (target_counts / total_counts) * 100
    # Calculate the percentage of each category in terms of the total number of observations
    total_percentages = (total_counts / total_counts.sum()) * 100
    # Sort the data for plotting
    total_counts_sorted = total_counts.sort_values(ascending=True)
    percentages_target_sorted = percentages_target.sort_values(ascending=True)

    # Create the subplot figure
    fig = make_subplots(rows=1, cols=2, subplot_titles=(
        f"Total number of {column_name}", f"Percentage of non-repayment for {column_name}"))

    for i, (value, percentage) in enumerate(total_counts_sorted.items()):
        count = total_counts[value]
        percentage = total_percentages[value]
        fig.add_trace(
            go.Bar(y=[value], x=[count], orientation='h', marker=dict(color=colors[i % len(colors)]),
                   text=f"{percentage:.2f}%"),
            row=1, col=1
        )

    for i, (value, percentage) in enumerate(percentages_target_sorted.items()):
        fig.add_trace(
            go.Bar(y=[value], x=[percentage], orientation='h', marker=dict(color=colors[i % len(colors)]), name=f"Percentage {value}",
                   text=f"{percentage:.2f}%"),
            row=1, col=2
        )

    # Update the layout and display the figure
    fig.update_layout(height=600, width=1200, showlegend=False,
                      title_text=f"Comparison of {column_name}")
    fig.update_traces(textposition='outside')

    fig.show()


# Example usage of the function
create_comparison_bar_charts(application_prev_train, 'NAME_CONTRACT_TYPE_y')

### Merging dataframe

Maintenant que nous aovns analyser les différents qui compose nos données nous allons aggréger ces données dans un seul et même dataframe

In [3]:
# Create a simple dataset with the train / test merge app
data = pd.concat([app_test, app_train], ignore_index=True)

In [ ]:
display(data.head())

In [4]:
# Check data
print('Train:' + str(app_test.shape))
print('Test:' + str(app_train.shape))
print('>>> Data:' + str(data.shape))

Train:(48744, 121)
Test:(307511, 122)
>>> Data:(356255, 122)


In [5]:
sum(data.SK_ID_CURR[data.TARGET.isna()] ==
    app_test.SK_ID_CURR)  # all is good

48744


A partir du fichier bureau.csv, il est possible d'extraire un historique sur les précédents crédits enregistrés par les clients. Il peut donc être intéressant d'enrichir l'échantillon avec ce type de données.

In [ ]:
display(bureau.head())
display(bureau.shape)

In [4]:
# Total number of previous credits taken by each customer
previous_loan_counts = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(
    columns={'SK_ID_BUREAU': 'PREVIOUS_LOANS_COUNT'})
previous_loan_counts.head()

,SK_ID_CURR,PREVIOUS_LOANS_COUNT
0,100001,7
1,100002,8
2,100003,4
3,100004,2
4,100005,3


In [5]:
# Merge this new column in our data sample
data = data.merge(previous_loan_counts, on='SK_ID_CURR', how='left')
data.shape

(356255, 123)

bureau_balance : bureau_balance.csv

In [ ]:
display(bureau_balance.head())
display(bureau_balance.shape)

In [6]:
# Monthly average balances of previous credits in Credit Bureau.
bureau_bal_mean = bureau_balance.groupby('SK_ID_BUREAU', as_index=False)[
    'MONTHS_BALANCE'].mean().rename(columns={'MONTHS_BALANCE': 'MONTHS_BALANCE_MEAN'})
bureau_bal_mean.head()

,SK_ID_BUREAU,MONTHS_BALANCE_MEAN
0,5001709,-48.0
1,5001710,-41.0
2,5001711,-1.5
3,5001712,-9.0
4,5001713,-10.5


In [8]:
bureau_full = bureau.merge(bureau_bal_mean, on='SK_ID_BUREAU', how='left')
bureau_full.drop('SK_ID_BUREAU', axis=1, inplace=True)
display(bureau_full.head())
display(bureau_full.shape)

,SK_ID_CURR,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,MONTHS_BALANCE_MEAN
0,215354,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,NaN
1,215354,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,NaN
2,215354,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,NaN
3,215354,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,NaN
4,215354,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,NaN


(1716428, 17)

In [9]:
bureau_mean = bureau_full.select_dtypes(include=[np.number]).groupby(
    'SK_ID_CURR', as_index=False).mean().add_prefix('PREV_BUR_MEAN_')
bureau_mean = bureau_mean.rename(
    columns={'PREV_BUR_MEAN_SK_ID_CURR': 'SK_ID_CURR'})

In [10]:
# Merge all this features with our data sample
data = data.merge(bureau_mean, on='SK_ID_CURR', how='left')
data.shape

(356255, 136)

previous_application

Vérification des valeurs de 'SK_ID_CURR' entre data et previous_application…

In [ ]:
display(previous_application.head())
display(previous_application.shape)

In [10]:
# Check
len(previous_application.SK_ID_CURR.isin(
    data.SK_ID_CURR)) == len(previous_application)

True

In [11]:
# Number of previous applications of the clients to Home Credit
previous_application_counts = previous_application.groupby('SK_ID_CURR',
                                                           as_index=False)['SK_ID_PREV'].count().rename(
    columns={'SK_ID_PREV': 'PREVIOUS_APPLICATION_COUNT'})
previous_application_counts.head()

,SK_ID_CURR,PREVIOUS_APPLICATION_COUNT
0,100001,1
1,100002,1
2,100003,3
3,100004,1
4,100005,2


In [12]:
# Merge this new column in our data sample
data = data.merge(previous_application_counts, on='SK_ID_CURR', how='left')
data.shape

(356255, 137)

credit_card_balance

In [ ]:
display(credit_card_balance.head())
display(credit_card_balance.shape)

In [13]:
credit_card_balance.drop('SK_ID_CURR', axis=1, inplace=True)

In [14]:
credit_card_balance_mean = credit_card_balance.groupby(
    'SK_ID_PREV', as_index=False).mean(numeric_only=True).add_prefix('CARD_MEAN_')
credit_card_balance_mean.rename(
    columns={'CARD_MEAN_SK_ID_PREV': 'SK_ID_PREV'}, inplace=True)

In [15]:
# Merge with previous_application
previous_application = previous_application.merge(
    credit_card_balance_mean, on='SK_ID_PREV', how='left')
previous_application.shape

(1670214, 57)

installments_payments

In [ ]:
display(installments_payments.head())
display(installments_payments.shape)

In [16]:
installments_payments.drop('SK_ID_CURR', axis=1, inplace=True)

In [17]:
install_pay_mean = installments_payments.groupby(
    'SK_ID_PREV', as_index=False).mean().add_prefix('INSTALL_MEAN_')
install_pay_mean.rename(
    columns={'INSTALL_MEAN_SK_ID_PREV': 'SK_ID_PREV'}, inplace=True)
install_pay_mean.shape

(997752, 7)

In [18]:
# Merge with previous_application
previous_application = previous_application.merge(
    install_pay_mean, on='SK_ID_PREV', how='left')
previous_application.shape

(1670214, 63)

POS_CASH_balance

In [ ]:
display(pos_cash_balance.head())
display(pos_cash_balance.shape)

In [19]:
pos_cash_balance.drop('SK_ID_CURR', axis=1, inplace=True)

In [20]:
POS_mean = installments_payments.groupby(
    'SK_ID_PREV', as_index=False).mean().add_prefix('POS_MEAN_')
POS_mean.rename(columns={'POS_MEAN_SK_ID_PREV': 'SK_ID_PREV'}, inplace=True)
POS_mean.shape

(997752, 7)

In [21]:
# Merge with previous_application
previous_application = previous_application.merge(
    POS_mean, on='SK_ID_PREV', how='left')
previous_application.shape

(1670214, 69)

previous_application

Retour sur previous_application pour assembles les lignes d'observation selon 'SK_ID_CURR'.

In [ ]:
display(previous_application.head())
display(previous_application.shape)

In [22]:
prev_appl_mean = previous_application.groupby('SK_ID_CURR', as_index=False).mean(
    numeric_only=True).add_prefix('PREV_APPL_MEAN_')
prev_appl_mean.rename(
    columns={'PREV_APPL_MEAN_SK_ID_CURR': 'SK_ID_CURR'}, inplace=True)
prev_appl_mean = prev_appl_mean.drop('PREV_APPL_MEAN_SK_ID_PREV', axis=1)

In [ ]:
display(prev_appl_mean.head())
display(prev_appl_mean.shape)

In [ ]:
# Reminder…
print('data shape', data.shape)

In [23]:
# Last merge with our data sample
data = data.merge(prev_appl_mean, on='SK_ID_CURR', how='left')
# data.set_index('SK_ID_CURR', inplace=True)
display(data.head())
display(data.shape)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,PREV_APPL_MEAN_INSTALL_MEAN_DAYS_INSTALMENT,PREV_APPL_MEAN_INSTALL_MEAN_DAYS_ENTRY_PAYMENT,PREV_APPL_MEAN_INSTALL_MEAN_AMT_INSTALMENT,PREV_APPL_MEAN_INSTALL_MEAN_AMT_PAYMENT,PREV_APPL_MEAN_POS_MEAN_NUM_INSTALMENT_VERSION,PREV_APPL_MEAN_POS_MEAN_NUM_INSTALMENT_NUMBER,PREV_APPL_MEAN_POS_MEAN_DAYS_INSTALMENT,PREV_APPL_MEAN_POS_MEAN_DAYS_ENTRY_PAYMENT,PREV_APPL_MEAN_POS_MEAN_AMT_INSTALMENT,PREV_APPL_MEAN_POS_MEAN_AMT_PAYMENT
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,-1664.000000,-1679.500000,7312.725000,7312.725000,1.250000,2.500000,-1664.000000,-1679.500000,7312.725000,7312.725000
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,-586.000000,-609.555556,6240.205000,6240.205000,1.111111,5.000000,-586.000000,-609.555556,6240.205000,6240.205000
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,-854.833333,-867.592593,16349.077917,13702.794792,1.050926,6.027778,-854.833333,-867.592593,16349.077917,13702.794792
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,-944.964286,-949.814286,7836.897982,7557.738339,1.038889,17.595238,-944.964286,-949.814286,7836.897982,7557.738339
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,-622.000000,-634.250000,11100.337500,11100.337500,1.000000,6.500000,-622.000000,-634.250000,11100.337500,11100.337500


(356255, 188)

## Feature enginering

Le "feature engineering" est le processus de transformation des données brutes en caractéristiques (features) utiles qui aident à améliorer la performance d'un modèle de machine learning. Les différents types incluent la création de variables (comme combiner des colonnes), la sélection de caractéristiques (choisir les variables les plus pertinentes), l'extraction de caractéristiques (comme l'analyse de composantes principales), et la transformation de variables (normalisation, standardisation, encodage de variables catégorielles).


**3 features extraites des précédentes étapes**

Pour rappel, les étapes précédentes consistaient uniquement à établir des liens entre nos fichiers, des fusions de table dans le but d'enrichir l'échantillon de travail. Ceci étant, avant de procéder au merging des éléments, on a pu facilement extraire 3 variables de moyenne et de comptage.
- PREVIOUS_LOANS_COUNT from bureau.csv: Nombre total des précédents crédits pris par chaque client
- MONTHS_BALANCE_MEAN from bureau_balance.csv: Solde moyen mensuel des précédents crédits
- PREVIOUS_APPLICATION_COUNT from previous_application.csv: Nombre de demandes antérieures des clients au crédit immobilier<br>

**Création de 4 nouvelles variables métiers**

Sans être expert en crédit bancaire, on peut assez facilement apporter quelques ratios explicatifs. D'autant plus qu'une veille parallèle permet de mieux comprendre les enjeux attendus. Voyons ci-dessous quelles features est-il pertinent d'intégrer.

- CREDIT_INCOME_PERCENT: Pourcentage du montant du crédit par rapport au revenu d'un client
- ANNUITY_INCOME_PERCENT: Pourcentage de la rente de prêt par rapport au revenu d'un client
- CREDIT_TERM: Durée du paiement en mois
- DAYS_EMPLOYED_PERCENT: Pourcentage des jours employés par rapport à l'âge du client


### Features domaines métier

Sans être expert en crédit bancaire, on peut assez facilement apporter quelques ratios explicatifs. D'autant plus qu'une veille parallèle permet de mieux comprendre les enjeux attendus. Voyons ci-dessous quelles features est-il pertinent d'intégrer.

- CREDIT_INCOME_PERCENT: Pourcentage du montant du crédit par rapport au revenu d'un client
- ANNUITY_INCOME_PERCENT: Pourcentage de la rente de prêt par rapport au revenu d'un client
- CREDIT_TERM: Durée du paiement en mois
- DAYS_EMPLOYED_PERCENT: Pourcentage des jours employés par rapport à l'âge du client

In [ ]:
# Before…
data.shape

In [ ]:
data['CREDIT_INCOME_PERCENT'] = data['AMT_CREDIT'] / data['AMT_INCOME_TOTAL']
data['ANNUITY_INCOME_PERCENT'] = data['AMT_ANNUITY'] / data['AMT_INCOME_TOTAL']
data['CREDIT_TERM'] = data['AMT_ANNUITY'] / data['AMT_CREDIT']
data['DAYS_EMPLOYED_PERCENT'] = data['DAYS_EMPLOYED'] / data['DAYS_BIRTH']

In [ ]:
# After…
data.shape

In [ ]:
# New Variables from features engineering
features_engin = ['PREVIOUS_LOANS_COUNT', 'MONTHS_BALANCE_MEAN', 'PREVIOUS_APPLICATION_COUNT',
                  'CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT', 'CREDIT_TERM', 'DAYS_EMPLOYED_PERCENT']

In [ ]:
# Display correlations with features engineering
print('Most Positive Correlations:\n', data.corr(numeric_only=True)
      ['TARGET'].sort_values().tail(15))
print("--------------------------")
print('Most Negative Correlations:\n', data.corr(numeric_only=True)
      ['TARGET'].sort_values().head(15))

### Polynomial Features

In [ ]:


# Make a new dataframe for polynomial features

poly_features = data[[
    'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'TARGET']]


# imputer for handling missing values
imputer = SimpleImputer(strategy='median')

poly_target = poly_features['TARGET']

poly_features = poly_features.drop(columns=['TARGET'])

# Need to impute missing values
poly_features = imputer.fit_transform(poly_features)
poly_features_test = imputer.transform(poly_features_test)


# Create the polynomial object with specified degree
poly_transformer = PolynomialFeatures(degree=3)

In [ ]:
# Train the polynomial features
poly_transformer.fit(poly_features)

# Transform the features
poly_features = poly_transformer.transform(poly_features)
poly_features_test = poly_transformer.transform(poly_features_test)
print('Polynomial Features shape: ', poly_features.shape)

In [ ]:
poly_transformer.get_feature_names_out(
    input_features=['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'])[:15]

In [ ]:


# Create a dataframe of the features
poly_features = pd.DataFrame(poly_features,
                             columns=poly_transformer.get_feature_names_out(['EXT_SOURCE_1', 'EXT_SOURCE_2',
                                                                             'EXT_SOURCE_3', 'DAYS_BIRTH']))

# Add in the target
poly_features['TARGET'] = poly_target

# Find the correlations with the target
poly_corrs = poly_features.corr()['TARGET'].sort_values()

# Display most negative and most positive
print(poly_corrs.head(10))
print(poly_corrs.tail(5))

In [ ]:
# Put test features into dataframe
poly_features_test = pd.DataFrame(poly_features_test,
                                  columns=poly_transformer.get_feature_names_out(['EXT_SOURCE_1', 'EXT_SOURCE_2',
                                                                                  'EXT_SOURCE_3', 'DAYS_BIRTH']))

# Merge polynomial features into training dataframe
poly_features['SK_ID_CURR'] = app_train['SK_ID_CURR']
app_train_poly = app_train.merge(poly_features, on='SK_ID_CURR', how='left')

# Merge polnomial features into testing dataframe
poly_features_test['SK_ID_CURR'] = app_test['SK_ID_CURR']
app_test_poly = app_test.merge(poly_features_test, on='SK_ID_CURR', how='left')

# Align the dataframes
app_train_poly, app_test_poly = app_train_poly.align(
    app_test_poly, join='inner', axis=1)

# Print out the new shapes
print('Training data with polynomial features shape: ', app_train_poly.shape)
print('Testing data with polynomial features shape:  ', app_test_poly.shape)

### Preprocessing des données

Il est nécessaire de commencer par la mise en place des données d'entrainement / test. On peut procéder en rappel avec les jeux de données application_train/test.

In [24]:
data_train = data[data['SK_ID_CURR'].isin(app_train.SK_ID_CURR)]
data_test = data[data['SK_ID_CURR'].isin(app_test.SK_ID_CURR)]

data_test = data_test.drop('TARGET', axis=1)

In [25]:
data_train.set_index('SK_ID_CURR', inplace=True)
data_test.set_index('SK_ID_CURR', inplace=True)

In [25]:
print('Training Features shape with categorical columns: ', data_train.shape)
print('Testing Features shape with categorical columns: ', data_test.shape)

Training Features shape with categorical columns:  (307511, 187)
Testing Features shape with categorical columns:  (48744, 186)


### Encodage des variables catégorielle

Une modèle de machine learning n'est pas capable d'interpréter les variables catégorielle. Pour pouvoir les utiliser dans notre modèle on doit encoder ces variables en nombre.Il existe deux méthode pour transformer ces valeur, soit le _Label Encoding_ ou le _one-hot-encoding_.Pour les colonnes ayant seulement deux valeur unique nous utiliseront la méthode du Label Encoding et pour les autres le one-hot-encoding.

### Label Encoding et One-Hot Encoding

In [26]:
# Create a label encoder object
le = LabelEncoder()
list_col_name = []
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            list_col_name.append(col)
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])

            # Keep track of how many columns were label encoded
            le_count += 1

print(f'{le_count} colonnes ont été encodées:{list_col_name}')

3 colonnes ont été encodées:['NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']


In [27]:
# one-hot encoding of categorical variables
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 243)
Testing Features shape:  (48744, 239)


### Alignement des datasets train et test

Le one-hot-encoding a créer plus de colonne dans le training data car certaines categories ne sont pas representer dans les données de test. Pour supprimer ces colonnes nous allons 'aligné' nos dataframes, tout en gardant la colonne 'TARGET' dans notre app_train.

In [28]:
train_labels = app_train['TARGET']

# Align the training and testing data, keep only columns present in both dataframes
app_train, app_test = app_train.align(app_test, join='inner', axis=1)

# Add the target back in
app_train['TARGET'] = train_labels

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 240)
Testing Features shape:  (48744, 239)


In [29]:
# Copy before imputation of missing values
train = data_train.copy()
test = data_test.copy()
train.shape, test.shape

((307511, 187), (48744, 186))

### Imputation des valeurs manquantes

Nous allons maintenant passé à l'imputation des valeurs manquantes dans notre jeu de données.

#### Choix de la méthode d'imputation 

Pour choisir la meilleurs méthode d'imputation nous avons sélectionner différentes méthode d'imputation (imputation par la moyenne,médiane,knn imputer). Pour ce test nous allons tester les modèles et évaluer grâce au MSE. On a choisi différents algorithme du plus complexe au plus simple pour voir si il y'a une amélioration significative. Pour ces test nous allons exécuter les mêmes test avec le même sample et de Cross Validation et la même metric d'évaluation.


#### Test Imputation par le KNN Imputer

In [30]:
# Reduce the size of the data and convert types to save memory
data_sample = app_train.sample(frac=0.1, random_state=42)
for col in data_sample.columns:
    if data_sample[col].dtype == np.float64:
        data_sample[col] = data_sample[col].astype(np.float32)
    if data_sample[col].dtype == np.int64:
        data_sample[col] = data_sample[col].astype(np.int32)

# Display the size of the sample
print(f"Sample size: {data_sample.shape}")

X = data_sample.drop('TARGET', axis=1)
y = data_sample['TARGET']

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Pipeline with KNNImputer and LinearRegression
pipeline = Pipeline([
    # Initial value for n_neighbors; will be adjusted by GridSearch
    ('imputer', KNNImputer(n_neighbors=5)),
    ('model', LinearRegression())
])

# Parameters for GridSearchCV
param_grid = {
    # Extending the search range up to 15
    'imputer__n_neighbors': list(range(3, 16))
}

# Setting up GridSearchCV and executing the Grid Search
grid_search = GridSearchCV(pipeline, param_grid, cv=5,
                           scoring='neg_mean_squared_error', verbose=3)
grid_search.fit(X_train, y_train)

# Best parameters found
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Evaluation on the test set with the best model
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)
test_score = mean_squared_error(y_test, predictions)
print(f"MSE on the test set: {test_score}")

# Accessing cross-validation results for each parameter configuration
cv_results = grid_search.cv_results_

# Preparing the DataFrame to store the results
results_df = pd.DataFrame(cv_results["params"])
for i in range(5):  # Assuming cv=5 as specified in GridSearchCV
    results_df[f'MSE Fold {i+1}'] = -cv_results[f'split{i}_test_score']
results_df['MSE Mean'] = -cv_results['mean_test_score']

# Adding the best parameter and the MSE on the test set to the DataFrame
results_df['Best Parameter'] = results_df['imputer__n_neighbors'] == best_params['imputer__n_neighbors']
results_df.loc['Mean / Total'] = results_df.mean(numeric_only=True)
results_df.at['Mean / Total', 'Best Parameter'] = test_score

# Saving the results to a CSV file
results_df.to_csv('grid_search_results.csv', index=False)

print("Results saved in 'grid_search_results.csv'.")

Résultats :<br>
**Meilleurs paramètres:** {'imputer__n_neighbors': 5}<br>

**MSE sur l'ensemble de test:** 0.07102997368792968

#### Test Imputation par SimpleImputer(Mean)

In [ ]:
# Sample a fraction of the data to reduce memory usage
data_sample = app_train.sample(frac=0.1, random_state=42)

# Convert data types to save memory
for col in data_sample.columns:
    if data_sample[col].dtype == np.float64:
        data_sample[col] = data_sample[col].astype(np.float32)
    if data_sample[col].dtype == np.int64:
        data_sample[col] = data_sample[col].astype(np.int32)

X = data_sample.drop('TARGET', axis=1)
y = data_sample['TARGET']

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)
# Assuming 'app_train' is your initial DataFrame
# Continuing with the already defined data_sample for the example

# Creating a baseline pipeline
baseline_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Imputation using the mean
    ('model', LinearRegression())  # Linear Regression model
])

# Defining the scoring metric as negative MSE
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Executing cross-validation and converting MSE scores to positive
cv_scores = cross_val_score(baseline_pipeline, X, y, cv=5, scoring=mse_scorer)
cv_scores_positive = -cv_scores

# Calculating the mean of the positive MSE scores
mean_cv_score = np.mean(cv_scores_positive)

# Printing the results
print("MSE for each cross-validation :", cv_scores_positive)
print("Mean MSE scores across cross-validations :", mean_cv_score)

# Creating a DataFrame to store the results
results_df = pd.DataFrame(cv_scores_positive, columns=['MSE'])
results_df.loc['Mean'] = mean_cv_score

# Saving the results to a CSV file
results_df.to_csv('cv_mse_results.csv', index_label='Validation Fold')

**Résultats** <br>
Validation Fold : 0 <br>
MSE: 0.07248985128343831 <br>
Validation Fold : 1 <br>
MSE: 0.06996261719078271 <br>
Validation Fold :2 <br>
MSE: 0.07037241326004501 <br>
Validation Fold : 3 <br>
MSE: 0.0701993874565854 <br>
Validation Fold :4 <br>
MSE: 0.07247808597927778 <br>
Moyenne MSE,0.07110047103402585 <br>

#### Test Imputation par SimpleImputer(Median)

**Résultats** <br>

Validation Fold :0<br>
MSE:0.07241020685855701<br>
Validation Fold :1<br>
MSE:0.06994843129380458<br>
Validation Fold :2<br>
MSE:0.07038380135291558<br>
Validation Fold :3<br>
MSE:0.07017891302627659<br>
Validation Fold :4<br>
MSE:0.07249469414473592<br>
Moyenne:0.07108320933525794<br>


#### Interprétation des résultats

Comme on peut le voir les résultats obtenu sont assez similaire on va choisir une imputation par la médiane car il obtient un meilleur résultats que par la moyenne et demande moins de temps d'éxécution que le KNN imputer.

#### Imputation par la médianne 